In [1]:
import os
import sys

import requests

import numpy as np

from astropy import units as u
from astropy import table
from astropy.coordinates import SkyCoord, EarthLocation, get_sun
from astropy.time import Time
from astropy.io import fits

from importlib import reload

%matplotlib inline
from matplotlib import style, pyplot as plt

Most of the initial steps are in 2018Oct-AAT

# Initial setup 

In [2]:
working_dir = os.path.abspath('..')

%cd ../../saga
sagadir = !pwd
sagadir = sagadir[0]
!python setup.py build
%cd $working_dir

/Users/erik/projects/saga/saga
running build
running build_py
/Users/erik/projects/saga/erik


In [3]:
sagalibdir = os.path.join(sagadir, 'build/lib')
print('Adding', sagalibdir, 'to the python path')
sys.path.insert(0, sagalibdir)

import SAGA as saga
from SAGA.database import external 
saga.__version__

Adding /Users/erik/projects/saga/saga/build/lib to the python path


'0.10.10'

In [4]:
saga_database = saga.Database(os.path.join(os.environ['HOME'],'Dropbox/SAGA/'))

saga_host_catalog = saga.HostCatalog(saga_database)
saga_object_catalog = saga.ObjectCatalog(saga_database)

In [5]:
hosts_to_load = 'aat_2018b'
host_id_label = 'ID_STR'
targethosts = saga_host_catalog.load(hosts_to_load)
targethosts[host_id_label] = saga_host_catalog.resolve_id(hosts_to_load, 'string')

# Generate AAT-appropriate target catalogs 

In [6]:
#saga_database['manual_targets_aat2018a'].local.path = 'aat_targets_oct2018/manual_targets.csv'

# saga_targets = saga.TargetSelection(saga_database, 
#                                     #manual_selected_objids='manual_targets_aat2018b',
#                                     gmm_parameters={'sdss':'gmm_parameters_no_outlier', 
#                                                     'des': 'gmm_parameters_no_outlier_no_u'}, 
#                                     version=2)
saga_targets = saga.TargetSelection(saga_database, manual_selected_objids='manual_targets_aat2018b')

acats = {}
tcats = {}

In [7]:
def get_gaia_guidestars(hostname, magrng=(12.5, 13.5),
                        matchmagrng=(16, 17), d_matchmag=1, matchtol0=1*u.arcsec,
                        verbose=True, neighbor_cut=30*u.arcsec, nmagdown=4.5):
    """
    `magrng` is the range of magnitudes to actually select on.  It's ~r-band, based on the
    Evans et al. 2018 r-to-G conversion
    """
    print = __builtin__.print
    if not verbose:
        print = lambda *args, **kwargs:None
        
    host = saga_host_catalog.load_single(hostname)
    obj_cat = saga_object_catalog.load(hostname)[0]
    omag = obj_cat['r_mag']
    gaia_cat = table.Table.read('external_catalogs/astrometric/{}_gaia.ecsv'.format(hostname))
    gaia_sc = SkyCoord(gaia_cat['ra'], gaia_cat['dec'])
    gmag = gaia_cat['phot_g_mean_mag']
    
    omsk = (matchmagrng[0]<omag)&(omag<matchmagrng[1])
    gmsk = ((matchmagrng[0]-d_matchmag)<gmag)&(gmag<(matchmagrng[1]+d_matchmag))
    oscmsk = obj_cat['coord'][omsk]
    gscmsk = gaia_sc[gmsk]
    
    idx, d2d, _ = oscmsk.match_to_catalog_sky(gscmsk)
    sepmsk = d2d < matchtol0
    
    dra = (oscmsk[sepmsk].ra - gscmsk[idx][sepmsk].ra).to(u.arcsec)
    ddec = (oscmsk[sepmsk].dec - gscmsk[idx][sepmsk].dec).to(u.arcsec)
    offset = np.mean(dra), np.mean(ddec)
    
    print("Object catalog to Gaia offset:", offset, 'from', np.sum(sepmsk), 
          'objects ({:.1%})'.format(np.sum(sepmsk)/len(sepmsk)))
    
    #this polynomial is from Evans et al. 2018 for the G to r conversion
    Gmr_coeffs = (-0.1856, 0.1579, 0.02738, -0.0550)
    Gmr = np.polyval(Gmr_coeffs[::-1], gaia_cat['bp_rp'])
    gaia_cat['g_as_r_mag'] = gmag - Gmr
    
    gmsk = (magrng[0] < gaia_cat['g_as_r_mag'])&(gaia_cat['g_as_r_mag'] < magrng[1])
    gstars = gaia_cat[gmsk]
    print('Found', len(gstars), 'Gaia guide stars')
    
    if neighbor_cut is not None:
        possible_neighbor_stars = gaia_cat[gaia_cat['g_as_r_mag'] < magrng[1]+nmagdown]
        nsc = SkyCoord(possible_neighbor_stars['ra'], possible_neighbor_stars['dec'])
        gsc = SkyCoord(gstars['ra'], gstars['dec'])
        idx, d2d, _ = gsc.match_to_catalog_sky(nsc, 2)
        neighbor_present = d2d < neighbor_cut
        print(np.sum(neighbor_present), 'Have a brightish neighbor.  Removing them.')
        gstars = gstars[~neighbor_present]

    tab = table.Table({
        'TargetName': gstars['source_id'],
        'RA': (u.Quantity(gstars['ra']) + offset[0]),
        'Dec': (u.Quantity(gstars['dec']) + offset[1]),
        'TargetType': np.repeat('F', len(gstars)),
        'Priority': np.repeat(9, len(gstars)),
        'Magnitude':gstars['g_as_r_mag'],
        '0': np.repeat(0, len(gstars)),
        'Notes': np.repeat('guide_gaia', len(gstars)),
    })
    tab.meta['dra'] = dra
    tab.meta['ddec'] = ddec
    tab.meta['offset'] = offset
    tab.meta['gstarcat'] = gstars
    return tab

In [8]:
def get_sdss_guidestars(hostname):
    host = saga_host_catalog.load_single(hostname)
    obj_cat = saga_object_catalog.load(hostname)[0]
    r = obj_cat['r_mag']
    msk = (12.5<r)&(r<14)&~obj_cat['is_galaxy']&(obj_cat['RHOST_ARCM']>15)
    starcat = obj_cat[msk]
    
    print('Found', len(starcat), 'SDSS guide stars')
    
    return table.Table({
        'TargetName': starcat['OBJID'],
        'RA': starcat['RA'],
        'Dec': starcat['DEC'],
        'TargetType': np.repeat('F', len(starcat)),
        'Priority': np.repeat(9, len(starcat)),
        'Magnitude': starcat['r_mag'],
        '0': np.repeat(0, len(starcat)),
        'Notes': np.repeat('guide', len(starcat)),
    })

In [9]:
from astropy.coordinates import Angle

def write_fld_file(target_catalog, host, obstime, fn, suffix=' master catalog'):
    hostname = host[host_id_label]
    target_catalog.write(fn,
                         delimiter=' ',
                         quotechar='"',
                         format='ascii.fast_commented_header',
                         overwrite=True,
                         formats={
                            'RA': lambda x: Angle(x, 'deg').wrap_at(360*u.deg).to_string('hr', sep=' ', precision=2), # pylint: disable=E1101
                            'Dec': lambda x: Angle(x, 'deg').to_string('deg', sep=' ', precision=2),
                            'Magnitude': '%.2f',
                         })
    with open(fn) as fh:
        content = fh.read()
    with open(fn, 'w') as fh:
        fh.write('LABEL ' + hostname + suffix + '\n')
        fh.write('UTDATE  {yr} {mo:02} {day:02}\n'.format(yr=obstime.datetime.year,
                                                        mo=obstime.datetime.month,
                                                        day=obstime.datetime.day))
        censtr = host['coord'].to_string('hmsdms', sep=' ', precision=2, alwayssign=True)
        fh.write('CENTRE  ' + censtr + '\n')
        fh.write('EQUINOX J2000.0\n')
        fh.write('# End of Header\n\n')
        fh.write(content.replace('"', ''))
        

def subsample_catalog(catalog, prilimits=None, maxflux=np.inf, maxguides=np.inf, maxsky=np.inf, verbose=True):
    """
    Subsamples the catalog, limiting the number in a given priority set
    
    ``prilimits`` should be a dictionary mapping priority number to the maximum number of
    objects in that priority.  If it is None, no limits will be used.
    """
    
    
    msks = [catalog['Notes']==notestr for notestr in ('Flux', 'Sky')]
    msks.append(np.array([n.lower().startswith('guide') for n in catalog['Notes']]))
    maxns = [maxflux, maxsky, maxguides]
    
    
    
    if prilimits is not None:
        for pri, maxn in prilimits.items():
            msks.append(catalog['Priority']==pri)
            maxns.append(maxn)
            
    idxs_to_rem = []
    for msk, maxn in zip(msks, maxns):
        ntorem = np.sum(msk) - maxn
        if ntorem >= 1:
            idxs = np.where(msk)[0]
            idxs_to_rem.append(np.random.permutation(idxs)[:ntorem])
    subcat = catalog.copy()
    if idxs_to_rem:
        del subcat[np.concatenate(idxs_to_rem)]
    
    if verbose:
        npris = dict(enumerate(np.bincount(subcat['Priority'][subcat['Notes']=='Targets'])))
        npris = {k:v for k,v in npris.items() if v>0}
        print('Nflux:', np.sum(subcat['Notes']=='Flux'), 
              'Nguide:', np.sum([n.lower().startswith('guide') for n in subcat['Notes']]), 
              'NSky', np.sum(subcat['Notes']=='Sky'))
        print('Targets in each priority:', npris)
        print('Total:', len(subcat))
        
    return subcat

In [10]:
def infer_radec_cols(table):
    raname = decname = None
    for cnm in table.colnames:
        if cnm.lower()=='ra':
            raname = cnm
        elif cnm.lower()=='dec':
            decname = cnm
    return raname, decname

In [11]:
def make_decals_viewer_cutouts(table, survey='sdss', ncols=3, zoom=15, size=120, namecol=None, dhtml=True):
    """
    Zoom of 15 is ~1"/pixel, so ~2' across with defaults
    """
    template_url = 'http://legacysurvey.org/viewer/jpeg-cutout/?ra={ra:.7}&dec={dec:.7}&zoom={zoom}&layer={layer}&size={size}'
    
    raname, decname = infer_radec_cols(table)
    
    entries = []
    for row in table:
        imgurl = template_url.format(ra=row[raname], dec=row[decname], layer=survey, size=size, zoom=zoom)
        viewurl = 'http://legacysurvey.org/viewer?ra={}&dec={}'.format(row[raname], row[decname])
        
        namestr = '' if namecol is None else (str(row[namecol]) + '<br>')
        entries.append('{}<a href="{}"><img src="{}"></a>'.format(namestr, viewurl, imgurl))
        
    entryrows = [[]]
    while entries:
        entry = entries.pop(0)
        if len(entryrows[-1]) >= ncols:
            entryrows.append([])
        entryrows[-1].append(entry)
    entryrows[-1].extend(['']*(ncols-len(entryrows[-1])))
    
    tabrows = ['<td>{}</td>'.format('</td><td>'.join(erow)) for erow in entryrows]

    htmlstr = """
    <table>
    <tr>{}</tr>
    </table>
    """.format('</tr>\n<tr>'.join(tabrows))

    if dhtml:
        from IPython import display
        return display.HTML(htmlstr)
    else:
        return htmlstr

In [12]:
def make_des_cutouts_file(table, copytoclipboard=True, showtable=False):
    raname, decname = infer_radec_cols(table)
    
    entries = []
    for row in table:
        entries.append('{:.5f},{:.5f}'.format(row[raname], row[decname]))
        
    htmlstr = """
    Paste the coords in "Enter values" at  
    <a href="https://des.ncsa.illinois.edu/easyweb/cutouts">the DES cutout service</a>,
    and change the size to 0.3 arcmin
    """
    tablestr = """<br>
    <table>
    <tr><td>{}</td></tr>
    </table>
    """.format('</td></tr>\n<tr><td>'.join(entries))
    if showtable:
        htmlstr += tablestr
    
    if copytoclipboard:
        import platform
        text = '\n'.join(entries)
        if platform.system() == 'Darwin':
            clipproc = os.popen('pbcopy', 'w')
            clipproc.write(text)
            clipproc.close()
            print('Copied to clipboard')
        elif platform.system() == 'Linux':
            clipproc = os.popen('xsel -i', 'w')
            clipproc.write(text)
            clipproc.close()
            print('Copied to clipboard')
        else:
            raise OSError("Not on a mac or linux, so can't use clipboard. ")
        
    from IPython import display
    return display.HTML(htmlstr)
    
def show_des_cutouts(table, jobname, username='eteq', ncols=3, namecol=None, force_size=(128, 128), dhtml=True):
    base_url = 'https://des.ncsa.illinois.edu'
    
    list_url = base_url + '/easyweb/static/workdir/{}/{}/list.json'.format(username, jobname)
    print(list_url)
    list_json = requests.get(list_url).json()
    img_urls = [base_url+img['name'] for img in list_json]
    
    if len(table) != len(img_urls):
        raise ValueError('table and job length do not match!')
    raname, decname = infer_radec_cols(table)
    
    sizestr = '' if force_size is None else ('height="{}" width="{}"'.format(force_size[1], force_size[0]))
    entries = []
    for row, imgurl in zip(table, img_urls):
        viewurl = 'http://legacysurvey.org/viewer?ra={}&dec={}'.format(row[raname], row[decname])
        
        namestr = '' if namecol is None else (str(row[namecol]) + '<br>')
        entries.append('{}<a href="{}"><img src="{}"{}></a>'.format(namestr, viewurl, imgurl, sizestr))
        
    entryrows = [[]]
    while entries:
        entry = entries.pop(0)
        if len(entryrows[-1]) >= ncols:
            entryrows.append([])
        entryrows[-1].append(entry)
    entryrows[-1].extend(['']*(ncols-len(entryrows[-1])))
    
    tabrows = ['<td>{}</td>'.format('</td><td>'.join(erow)) for erow in entryrows]

    htmlstr = """
    <table>
    <tr>{}</tr>
    </table>
    """.format('</tr>\n<tr>'.join(tabrows))

    if dhtml:
        from IPython import display
        return display.HTML(htmlstr)
    else:
        return htmlstr


# Configuration File-Generation Templates 

## nsaxxxx

In [ ]:
idnum = xxxx
fieldnum = NN

saga_object_catalog.build_and_write_to_database('NSA{}'.format(idnum), overwrite=fieldnum>1)

In [ ]:
host = targethosts[targethosts['NSAID'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_sdss_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

In [ ]:
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))], 
                           survey='sdss',ncols=6,namecol='TargetName')

In [ ]:
guides_to_rem = [int(i) for i in 'nnnn nnn nnnn'.split()]
writecat.remove_rows((writecat['Notes']=='Guide')&(np.in1d(writecat['TargetName'], guides_to_rem)))
np.sum((writecat['Notes']=='Guide'))

In [ ]:
make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='sdss', ncols=6, 
                           zoom=16, namecol='TargetName')

In [ ]:
skys_to_rem = [int(i) for i in 'n n nn nn'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

In [ ]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

In [ ]:
!scp $fldfn visitor8@aatlxh:~/octDD/

## pgcxxxx

In [ ]:
idnum = xxxx
fieldnum = NN

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

In [ ]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

In [ ]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

In [ ]:
make_des_cutouts_file(acat[acat['Notes']=='sky'])

In [ ]:
show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

In [ ]:
skys_to_rem = [int(i) for i in <'n n nn'>.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

In [ ]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

In [ ]:
!scp $fldfn visitor8@aatlxh:~/octDD/

# Night 1

In [24]:
import aat

obs_start_date = Time('2018-8-3') 

## pgc64427

In [14]:
idnum = 64427
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/03 13:23:56] Base catalog v2 for pgc64427 already exists (./base_catalogs/base_v2_pgc64427.fits.gz). (1/1)


In [15]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list').pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc64427
# of flux stars = 616
# of sky fibers = 100
# of Priority=1 targets = 92
# of Priority=2 targets = 75
# of Priority=3 targets = 31
# of Priority=4 targets = 49
# of Priority=5 targets = 3
# of Priority=6 targets = 3
# of Priority=7 targets = 1
# of Priority=8 targets = 8
# of Priority=9 targets = 716
Object catalog to Gaia offset: (<Angle -0.05738712 arcsec>, <Angle 0.13878442 arcsec>) from 2493 objects (73.6%)
Found 348 Gaia guide stars
205 Have a brightish neighbor.  Removing them.


1121

In [22]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

6472750671259845248,6473545652526720896,6473796062002851328,6473429035574240640,6473489779295023104,6473764489700090496
6473821660007930112,6473830941434283904,6473841043197506816,6473624198888872448,6473396565621518336,6473527854182187520
6473477139207971328,6473740540962574720,6473582383087121664,6473701577019165440,6473856397704538368,6473613375571142656
6473425462161419136,6473424809326403584,6473614200204862720,6473625019225213952,6473424912405613952,6472657693807885824


In [17]:
make_des_cutouts_file(acat[acat['Notes']=='sky'])

Copied to clipboard


In [19]:
show_des_cutouts(acat[acat['Notes']=='sky'], '82f29aba-9bce-425a-8448-94fb8c86d7b1', ncols=8, namecol='TargetName')

https://des.ncsa.illinois.edu/easyweb/static/workdir/eteq/82f29aba-9bce-425a-8448-94fb8c86d7b1/list.json


0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [20]:
skys_to_rem = [int(i) for i in '1 3 4 8 11 12 13 15 17 23 24 28 29 31 32 34 39 40 42 48 50 55 56 60 61 73 74 75 80 82 83 85 87 88 90 93 94'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

63

In [25]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 63
Targets in each priority: {1: 92, 2: 75, 3: 31, 4: 49, 5: 3, 6: 3, 7: 1, 8: 8}
Total: 430
Writing aat_targets_oct2018/pgc64427_1.fld


262

In [27]:
!scp $fldfn visitor8@aatlxh:~/oct03/

pgc64427_1.fld                                100%   24KB  23.8KB/s   00:00    


## pgc67663

In [105]:
idnum = 67663
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=True)

[10/03 17:13:42] [WARNING] Not found: SDSS catalog for pgc67663.
[10/03 17:13:43] [WARNING] Not found: DECALS catalog for pgc67663.
[10/03 17:13:43] Use DES to build base catalog v2 for pgc67663 (1/1)


[10/03 17:14:06] Write base catalog to ./base_catalogs/base_v2_pgc67663.fits.gz


In [108]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list').pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc67663
21h56m09.6s -49d31m19.2s
# of flux stars = 209
# of sky fibers = 100
# of Priority=1 targets = 558
# of Priority=2 targets = 82
# of Priority=3 targets = 22
# of Priority=4 targets = 50
# of Priority=5 targets = 138
# of Priority=6 targets = 76
# of Priority=7 targets = 7
# of Priority=8 targets = 11
# of Priority=9 targets = 309
Object catalog to Gaia offset: (<Angle -0.12399641 arcsec>, <Angle 0.14305041 arcsec>) from 1117 objects (22.2%)
Found 187 Gaia guide stars
63 Have a brightish neighbor.  Removing them.


1377

In [47]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

6560619925403096960,6559378752870600960,6560672908119653632,6559378581071910784,6559166409687563520,6559099232102507648
6559195203148360448,6560624155947819392,6559403354443336704,6559434278207895680,6559186677636635776,6558340019324117760
6559423764127892480,6559410638707893120,6560698750939899392,6559155689449173120,6560690362866747776,6559276807526995200
6560743933995937536,6559066251050018944,6558981519935088000,6559371228087870208,6560726616687707392,6559078380037754752


In [48]:
make_des_cutouts_file(acat[acat['Notes']=='sky'])

Copied to clipboard


In [49]:
show_des_cutouts(acat[acat['Notes']=='sky'], '80012d2a-403a-4636-8047-bb226a30760a', ncols=8, namecol='TargetName')

https://des.ncsa.illinois.edu/easyweb/static/workdir/eteq/80012d2a-403a-4636-8047-bb226a30760a/list.json


0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [50]:
skys_to_rem = [int(i) for i in '0 2 9 11 14 17 18 20 22 23 26 28 33 34 38 39 40 41 42 43 45 47 49 50 51 60 61 62 64 65 67 68 72 75 76 78 79 80 83 85 86 91 94 96 97 98'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

54

In [51]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 54
Targets in each priority: {1: 558, 2: 82, 3: 22, 4: 50, 5: 138, 6: 76, 7: 7, 8: 11}
Total: 1103
Writing aat_targets_oct2018/pgc67663_1.fld


944

In [52]:
!scp $fldfn visitor8@aatlxh:~/oct03/

pgc67663_1.fld                                100%   60KB  60.1KB/s   00:00    


In [59]:
cut_cat = subsampledcat[subsampledcat['Priority']==1]
make_des_cutouts_file(cut_cat[np.random.permutation(len(cut_cat))][:50])

Copied to clipboard


WARNING: the above ones were corrupted by an old version of the base catalog and should be re-generated

## pgc1952

In [184]:
idnum = 1952
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/03 23:09:03] Base catalog v2 for pgc1952 already exists (./base_catalogs/base_v2_pgc1952.fits.gz). (1/1)


In [185]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc1952
00h32m02.112s -64d15m12.24s
# of flux stars = 141
# of sky fibers = 100
# of Priority=1 targets = 490
# of Priority=2 targets = 109
# of Priority=3 targets = 10
# of Priority=4 targets = 50
# of Priority=5 targets = 115
# of Priority=6 targets = 62
# of Priority=7 targets = 42
# of Priority=8 targets = 5
# of Priority=9 targets = 241
Object catalog to Gaia offset: (<Angle -0.36644631 arcsec>, <Angle 0.07111843 arcsec>) from 921 objects (72.6%)
Found 171 Gaia guide stars
46 Have a brightish neighbor.  Removing them.


1249

In [186]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

4708194728138803584,4900679459944794752,4708224208794242688,4708167927542912512,4900522367220988160,4708465482876995712
4900604074678797568,4708229500194002432,4708501148285313408,4900691520213008640,4708190914207846400,4708380064567371776
4708404150743953408,4900660218491329408,4708436586336930432,4900635990579864704,4900806178659741056,4708270319563218688
4708387967307187584,4708510459774392192,4900632451526689024,4708367660701955328,4900661695960072960,4900772506116151552


In [188]:
# Not necessary due to des now being in the decals viewer
#make_des_cutouts_file(acat[acat['Notes']=='sky'])
#show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')
make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [189]:
skys_to_rem = [int(i) for i in '1 5 9 11 15 16 18 20 22 24 27 29 35 41 43 50 54 55 58 59 60 61 62 65 67 69 71  75 76 77 79 90 91 93 96 97 98'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

63

In [190]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 63
Targets in each priority: {1: 490, 2: 109, 3: 10, 4: 50, 5: 115, 6: 62, 7: 42, 8: 5}
Total: 1051
Writing aat_targets_oct2018/pgc1952_1.fld


883

In [191]:
!scp $fldfn visitor8@aatlxh:~/oct03/

pgc1952_1.fld                                 100%   56KB  56.2KB/s   00:00    


# Scratch/working space 

## Check fibers from list file (mostly for guidestar when on sky) 

In [187]:
lt, sc, _ = aat.load_lis_file('aat_targets_oct2018/nsa13927_1_p0.lis')
lt['ra'] = sc.ra.deg
lt['dec'] = sc.dec.deg

In [172]:
lt[lt['codes']=='F']

fibnums,ids,ras,decs,codes,pris,mags,comments,ra,dec
int64,str19,str12,str12,str1,int64,float64,str7,float64,float64
50,1237651752400322581,12:05:30.220,+01:17:11.74,F,9,13.7,Guide,181.37591666666665,1.2865944444444444
150,1237651735220191273,12:01:01.170,+01:36:51.22,F,9,12.73,Guide,180.254875,1.6142277777777778
200,1237651735756996625,12:00:13.470,+01:57:34.90,F,9,13.37,Guide,180.05612499999998,1.9596944444444444
250,1237651754010345584,12:00:27.590,+02:33:38.73,F,9,13.62,Guide,180.1149583333333,2.5607583333333332
300,1237651736831000598,12:02:52.900,+02:45:30.57,F,9,13.29,Guide,180.72041666666664,2.7584916666666666
350,1237651754010935313,12:05:18.440,+02:34:09.82,F,9,13.01,Guide,181.32683333333333,2.5693944444444443
400,1237651735757651990,12:06:19.120,+01:55:08.36,F,9,13.91,Guide,181.57966666666664,1.9189888888888886


In [177]:
make_decals_viewer_cutouts(lt[lt['codes']=='F'], 'sdss', namecol='fibnums', ncols=4)

50,150,200,250
300,350,400,


In [176]:
make_decals_viewer_cutouts(lt[lt['codes']=='F'], 'unwise-w1w2', namecol='mags', ncols=4, zoom=17)

13.7,12.73,13.37,13.62
13.29,13.01,13.91,


### Double-check some of the targeting numbers 

In [132]:
score_bins  = np.array([1, 100, 150, 180, 200, 300, 400, 500, 600, 700, 800, 900, 1000])-1

count_tab = table.Table()
count_tab.add_column(table.Column(name='name', dtype='U10'))
for b1, b2 in zip(score_bins[:-1], score_bins[1:]):
    count_tab.add_column(table.Column(name='bin_{}to{}'.format(b1, b2), dtype=int))

for nm, cat in tcats.items():
    nbins = np.histogram(cat['TARGETING_SCORE'], score_bins)[0]
    count_tab.add_row([nm] + list(nbins))
    
count_tab.pprint(max_width=1000)

  name   bin_0to99 bin_99to149 bin_149to179 bin_179to199 bin_199to299 bin_299to399 bin_399to499 bin_499to599 bin_599to699 bin_699to799 bin_799to899 bin_899to999
-------- --------- ----------- ------------ ------------ ------------ ------------ ------------ ------------ ------------ ------------ ------------ ------------
pgc64427         0           2            0            6            1            3            3           49           31           75           92         2932
pgc67663         0          11            0            0            7           76          138           50           22           82          558         1636


In [ ]:
# in each bin
 host_id  100 150 180 200 300 400 500 600 700 800
--------- --- --- --- --- --- --- --- --- --- ---
 pgc64427   2   0   7  28   5   7  48  36 182 128
 pgc67663  11   0   0 109 102 158  50  29 130 785

# in each bin
 host_id  100 150 180 200 300 400 500 600 700 800
--------- --- --- --- --- --- --- --- --- --- ---
  pgc1952   5   1   0  48  63 118  50  10 128 503
  pgc2052   8   0  22  29  50  67  48   9  23 756
  pgc9747   8   1   0  34  62 182  50   6  44 783
 pgc10965   4   0   0  39  92 126  50  22  76 667
nsa133355   6   1   0  61  90 124  50  30  96 620
 pgc64427   2   0   7  28   5   7  48  36 182 128
 pgc66934   2   1   1  12  11  22  50  28 159 224
 pgc67146   2   0   0  29  27  52  50  17 102 459
 pgc67663  11   0   0 109 102 158  50  29 130 785
 pgc67817   2   2   5  24   8  12  50  21  61 263
 pgc70094  17   2   1  27  75 227  50  18  81 948
 pgc71729   6   0   0  45  68 179  50  24 117 817

In [169]:
saga.targets.assign_targeting_score_v2?

Signature: saga.targets.assign_targeting_score_v2(base, manual_selected_objids=None, gmm_parameters=None, ignore_specs=False, debug=False, n_random=50, seed=123, remove_lists=None, low_priority_objids=None, **kwargs)
Docstring:
Last updated: 09/05/2018
 100 Human selection and Special targets
 150 satellites without AAT/MMT specs
 180 low-z (z < 0.05) but ZQUALITY = 2
 200 within host,  r < 17.77, gri/grz cuts
 291 within host,  r < 21, very low SB (applied to DES only)
 300 within host,  r < 20.75, high p_GMM or GMM outliers, low SB, gri/grz cuts
 400 within host,  r < 20.75, high-priority, low SB, gri/grz cuts
 500 within host,  r < 20.75, gri/grz cuts, random selection of 50
 600 very high p_GMM, low SB
 700 outwith host, r < 17.77
 800 within host,  r < 20.75, gri/grz cuts, everything else
 900 outwith host, r < 20.75, gri/grz cuts
1000 everything else
1100 Not in gri/grz cut
1200 Not galaxy
1300 Not clean
1350 Removed by hand
1400 Has spec already
File:      ~/projects/saga/saga/b

In [178]:
tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
tcats[hostname + '_2'] = tcat

In [182]:
score_bins  = np.array([1, 100, 150, 180, 200, 300, 400, 500, 600, 700, 800, 900, 1000])-1

count_tab = table.Table()
count_tab.add_column(table.Column(name='name', dtype='U10'))
for b1, b2 in zip(score_bins[:-1], score_bins[1:]):
    count_tab.add_column(table.Column(name='bin_{}to{}'.format(b1, b2), dtype=int))

for nm, cat in tcats.items():
    nbins = np.histogram(cat['TARGETING_SCORE'], score_bins)[0]
    count_tab.add_row([nm] + list(nbins))
    
count_tab.pprint(max_width=1000)

   name    bin_0to99 bin_99to149 bin_149to179 bin_179to199 bin_199to299 bin_299to399 bin_399to499 bin_499to599 bin_599to699 bin_699to799 bin_799to899 bin_899to999
---------- --------- ----------- ------------ ------------ ------------ ------------ ------------ ------------ ------------ ------------ ------------ ------------
  pgc64427         0           2            0            6            1            3            3           49           31           75           92         2932
  pgc67663         0          11            0            0            7           76          138           50           22           82          558         1636
pgc67663_2         0          11            0            0          109          102          158           50           29          130          785         1799
